# 데이터 읽기 및 전처리

In [11]:
pip install -U sentence-transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=841ed9c396604cf87a641a0bd9ca778802f270d12edda46f1f60073400c5907e
  Stored in directory: c:\users\noa\appdata\local\pip\cache\wheels\52\19\88\6625593382e23a926740e6fcee0f2df0a0de25766094842a28
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -e .

Note: you may need to restart the kernel to use updated packages.


ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: C:\Users\NoA\NLU


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 999) # pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 999) # pd.options.display.max_columns = 999
pd.set_option('display.width', 1000)

# 한글폰트 적용
plt.rcParams['font.family'] = 'Malgun Gothic'

In [6]:
absolute_file_address = input("")

# read text data set
text_set = pd.read_csv(absolute_file_address)
text_set = text_set.fillna("")
text_set

all_url_title_text_20211203103401.csv


,url,title,text
0,https://blog.naver.com/apfjtzzz/222354696565,충남대 맛집 :: 삼각김밥의 클라쓰를 바꾼 강다짐,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​남편은 예전부터 편의점 ...
1,https://blog.naver.com/fa1772/222377394130,취향저격 충남대 맛집 리스트,\n\n\n\n\n최근 동창 친구와 약속이 있어서 충남대 맛집에 다녀왔는데요품질 좋...
2,https://blog.naver.com/1457dudgus/222565835669,"대전 궁동 맛집, 워터에이징 궁동고기집 충남대맛집 ""투더돼지""","\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n궁동맛집 ""투더돼지""\n\..."
3,https://blog.naver.com/help1207/222548639407,[ 대전 충남대맛집 ] [ 부쳐샵 충남대점 ] 유성구 수제버거 전문점,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n부쳐샵 충남대점\n\n\n...
4,https://blog.naver.com/sweets9/222548166927,[대전/봉명동] 데이트하기 좋은 양고기가 맛있는 충남대 맛집 진1926 봉명점 / ...,\n\n\n\n\n​​충남대 근처 진1926 에서 데이트했어용 :) 히히다찌석도 있...
...,...,...,...
4769,https://blog.naver.com/kfs97/221734966254,[칠곡맛집/닭개장맛집]호동이국밥,\n\n\n\n\n\n\n\n\n호동이국밥\n대구광역시 북구 동천동 912-1\n\...
4770,https://blog.naver.com/free6568/221779346538,인천예술회관맛집 호동이갈비살 인천점 저렴한 구월동 고기집,\n\n\n\n\n안녕하세요? 썅이네 다락방 빵떡맘입니다. ​구월동에는 맛집이 많아...
4771,https://blog.naver.com/lsk929292/222520018600,김해한우맛집 호동육회에서 육사시미와육회 반반세트 배민에서 주문해서 먹은 후기 ^^,\n\n\n\n\n 깔깔마녀 어제 아들 논산 훈련소에 데려다 주고 너무 펑펑 울어서...
4772,https://blog.naver.com/kertod331/222513981616,안동정하동맛집 술과 식사가 가능한 호동이 감자탕,\n\n\n\n\n다들 명절 잘 보내고 계신가요?저는 오늘 출근했습니다.. 다들 금...


In [7]:
text_set.shape

(4774, 3)

In [8]:
text_set = text_set.astype({'text':'string', 'url':'string'})
text_set.dtypes

url      string
title    object
text     string
dtype: object

# 데이터 분석 실행

In [9]:
import copy
import openpyxl
import csv

import re
from sentence_transformers import SentenceTransformer, util

f = open('submission.csv', 'w', newline='')
wr = csv.writer(f)
# 과도한 정보 표기 , 고유 단어 사용, 고료 표기, 유사한 문장 사용, 유사한 문장 사용
wr.writerow(["url", "use_tmi_words_value", "similar_sentence_value", "commissional_words_value"])

# Sentence-Transformers 패키지의 RoBERTa 알고리즘 사용
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

individual_text_set = list()
for text in text_set['text']:
    lines = text.split('\n')
    lines = [v for v in lines if v]
            
    individual_text_set = individual_text_set + [v for v in lines if len(v) > 5]
    
embeddings1 = model.encode(individual_text_set, convert_to_tensor=True)

In [10]:
analysis_count = 0
while analysis_count < len(text_set):
    
    ### 전처리 ###
    print("Analysis position:", analysis_count, "->", analysis_count+5)
    
    if analysis_count+5 < len(text_set)-1:
        text_sub_set = text_set[analysis_count : analysis_count+5]
    else:
        text_sub_set = text_set[analysis_count : len(text_set)-1]
        
    
    # matrix(DataFrame)에서 text데이터를 가져온다.
    text_list = text_sub_set['text']
    url_list = text_sub_set['url']
    url_list = url_list.tolist()
    
    
    
    # text 분할
    individual_text_sub_set = list()
    individual_text_sub_set_size = list()
    for text in text_list:
        lines = text.split('\n')
        lines = [v for v in lines if v]
        lines = [v for v in lines if len(v) > 5]
        
        if(len(lines) == 0):
            lines.append("NONE")
            
        individual_text_sub_set = individual_text_sub_set + lines
        individual_text_sub_set_size.append(len(lines))
        
        
    
    
    ### 무의미한 문장 찾기 ###
    
    use_tmi_words_value = np.zeros(shape=(len(text_list),), dtype=np.int64)

    # 주소
    address_keywords = ["인천", "서울", "경기", "강원", "충청", "충남", "세종", "충북", "대전", "경상", "경북", "경남", "대구", "전라", "전남", "전북", "울산", "부산"]
    # 전화번호
    phoneNumRegex = re.compile(r'((\d{2}|\(\d{2}\)|\d{3}|\(\d{3}\))?(|-|\.)?(\d{3}|\d{4})(\s|-|\.)(\d{4}))')
    # 불규칙 키워드 (피드백 조정)
    irregular_keywords = ['수상', '대회', '출연', '전화번호', '전화', '번호', '운영시간', '운영 시간', '영업시간', '영업 시간']



    dest_keywords = address_keywords + irregular_keywords
    for i in range(0, len(text_list)):
        for line in text_list.tolist()[i].split('\n'):
            if(line == "\n"):
                continue

            for keyword in dest_keywords:
                if(line.find(keyword) != -1):
                    use_tmi_words_value[i] = use_tmi_words_value[i] + 1

            use_tmi_words_value[i] = use_tmi_words_value[i] + len(phoneNumRegex.findall(line))
            
    print("use_tmi_words_value:", use_tmi_words_value)
    
    
    
    
    ### 문장 유사도 분석 ###
    
    similar_sentence_value = np.zeros(shape=(len(text_list),), dtype=np.int64)        

    embeddings2 = model.encode(individual_text_sub_set, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings2, embeddings1) # 코사인 유사도 공식
    
    
    for index in range(0, len(individual_text_sub_set)):
        if(individual_text_sub_set[index] == "NONE"):
            continue
        
        temp = cosine_scores[index]
        
        for i in temp.argsort(descending=True):
            if(cosine_scores[index][i] < 0.95):
                break
                
                
            
            temp_pos = individual_text_sub_set_size[0]
            for l in range(0, len(individual_text_sub_set_size)):
                if(index < temp_pos):
                    similar_sentence_value[l] = similar_sentence_value[l] + 1
                    break
                   
                else:
                   if(l+1 < len(individual_text_sub_set_size)):
                       temp_pos = temp_pos + individual_text_sub_set_size[l+1]
                    
    print("similar_sentence_value", similar_sentence_value)
    
    
    
    
    ### 공정위 문구 분석 ###
    
    commissional_words = ["협찬", "고료", "광고", "후원", "원고", "지원", "제공", "업체", "서비스"]

    commissional_words_value = np.zeros(shape=(len(text_list),), dtype=np.int64)

    for index in range(0, len(text_list)):
        for word in commissional_words:
            if(len(text_list) == 0):
                continue
            if(text_list.tolist()[index].find(word) != -1):
                commissional_words_value[index] = 1

    print("commissional_words_value:", commissional_words_value)
    
    for row in range(0, 5):
        if(len(use_tmi_words_value) <= row):
            break
        wr.writerow([url_list[row], use_tmi_words_value[row], similar_sentence_value[row], commissional_words_value[row]])
        
        
    analysis_count = analysis_count+5
    print("\nData cast")
    
f.close()

print("Analysis finish")

Analysis position: 0 -> 5
use_tmi_words_value: [12 36  8 15  9]
similar_sentence_value [ 973 5517  816 2263  719]
commissional_words_value: [0 1 1 1 1]

Data cast
Analysis position: 5 -> 10
use_tmi_words_value: [ 5 11  3  9 11]
similar_sentence_value [2465 4371   77 2378  214]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 10 -> 15
use_tmi_words_value: [23  9  6 14  8]
similar_sentence_value [2587 1429  258 2210 1958]
commissional_words_value: [0 1 0 1 1]

Data cast
Analysis position: 15 -> 20
use_tmi_words_value: [12 13 16 10 11]
similar_sentence_value [570 765 451 732 163]
commissional_words_value: [0 0 0 1 0]

Data cast
Analysis position: 20 -> 25
use_tmi_words_value: [ 4 11  5  3  8]
similar_sentence_value [1614 2058  266  161 3944]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 25 -> 30
use_tmi_words_value: [9 3 0 3 7]
similar_sentence_value [ 190  393   52  281 1698]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 30 -

similar_sentence_value [ 46 194 242 452 208]
commissional_words_value: [0 1 1 0 0]

Data cast
Analysis position: 255 -> 260
use_tmi_words_value: [7 0 1 0 4]
similar_sentence_value [584   0   3   0 966]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 260 -> 265
use_tmi_words_value: [13  0  0  0  0]
similar_sentence_value [6954    0    0    0    0]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 265 -> 270
use_tmi_words_value: [5 7 4 3 0]
similar_sentence_value [   6   78  987 3153    0]
commissional_words_value: [0 1 1 1 0]

Data cast
Analysis position: 270 -> 275
use_tmi_words_value: [15  0  7  3  5]
similar_sentence_value [  46    0  898  912 1490]
commissional_words_value: [0 0 1 1 0]

Data cast
Analysis position: 275 -> 280
use_tmi_words_value: [1 8 4 2 6]
similar_sentence_value [ 322  284  207 1986  523]
commissional_words_value: [0 0 1 0 0]

Data cast
Analysis position: 280 -> 285
use_tmi_words_value: [10  0 24  8  8]
similar_sentence_value 

similar_sentence_value [  57  940  116  226 1802]
commissional_words_value: [1 1 1 1 1]

Data cast
Analysis position: 505 -> 510
use_tmi_words_value: [1 6 2 2 1]
similar_sentence_value [ 199  938 1291 1274 1060]
commissional_words_value: [0 1 1 0 0]

Data cast
Analysis position: 510 -> 515
use_tmi_words_value: [1 2 5 1 1]
similar_sentence_value [ 352  153 1509  255  114]
commissional_words_value: [1 0 0 1 0]

Data cast
Analysis position: 515 -> 520
use_tmi_words_value: [2 7 0 0 2]
similar_sentence_value [ 111  689    0    0 1687]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 520 -> 525
use_tmi_words_value: [2 1 5 5 4]
similar_sentence_value [2921  633  493  329  543]
commissional_words_value: [0 0 1 0 0]

Data cast
Analysis position: 525 -> 530
use_tmi_words_value: [6 6 8 0 3]
similar_sentence_value [ 594  150  121    0 1786]
commissional_words_value: [0 1 0 0 0]

Data cast
Analysis position: 530 -> 535
use_tmi_words_value: [ 7 13 17  8  7]
similar_sentence_value 

similar_sentence_value [3395 1544  257 1308    0]
commissional_words_value: [1 1 0 1 0]

Data cast
Analysis position: 755 -> 760
use_tmi_words_value: [6 4 5 7 6]
similar_sentence_value [297 896  74 127 315]
commissional_words_value: [0 1 0 0 0]

Data cast
Analysis position: 760 -> 765
use_tmi_words_value: [ 2  9 10  0  8]
similar_sentence_value [  40 2800 1953    0  669]
commissional_words_value: [0 0 1 0 1]

Data cast
Analysis position: 765 -> 770
use_tmi_words_value: [4 0 0 6 0]
similar_sentence_value [ 688    0    0 2788    0]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 770 -> 775
use_tmi_words_value: [ 10   5   7  40 197]
similar_sentence_value [ 453  152  227  583 5838]
commissional_words_value: [1 1 1 1 1]

Data cast
Analysis position: 775 -> 780
use_tmi_words_value: [0 5 7 8 3]
similar_sentence_value [   0 2548  892  116  206]
commissional_words_value: [0 1 1 1 1]

Data cast
Analysis position: 780 -> 785
use_tmi_words_value: [3 7 6 3 4]
similar_sentence_v

similar_sentence_value [1032  516  163  526  245]
commissional_words_value: [1 0 0 0 1]

Data cast
Analysis position: 1005 -> 1010
use_tmi_words_value: [ 3  8  2  2 43]
similar_sentence_value [ 214 4692  226  587 3769]
commissional_words_value: [1 0 1 0 0]

Data cast
Analysis position: 1010 -> 1015
use_tmi_words_value: [45 44 42 44 37]
similar_sentence_value [4759 6461 3222 1373  923]
commissional_words_value: [1 0 1 1 1]

Data cast
Analysis position: 1015 -> 1020
use_tmi_words_value: [38  9 44 47  9]
similar_sentence_value [4427   56 2017 8042 2573]
commissional_words_value: [1 0 1 0 0]

Data cast
Analysis position: 1020 -> 1025
use_tmi_words_value: [ 4  5 44 48 34]
similar_sentence_value [1207 1053 4235 2414 1166]
commissional_words_value: [0 0 1 1 1]

Data cast
Analysis position: 1025 -> 1030
use_tmi_words_value: [29 36 43 35 18]
similar_sentence_value [2073 5105 4468 8088  335]
commissional_words_value: [0 1 1 1 0]

Data cast
Analysis position: 1030 -> 1035
use_tmi_words_value: [51

...

similar_sentence_value [ 435  832  181  115 2254]
commissional_words_value: [1 0 0 1 0]

Data cast
Analysis position: 3750 -> 3755
use_tmi_words_value: [ 8  9 10  3  5]
similar_sentence_value [1820 1629 1707  526  411]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 3755 -> 3760
use_tmi_words_value: [11  1 10 13  1]
similar_sentence_value [1216  115 1165 1198 1010]
commissional_words_value: [0 0 0 1 0]

Data cast
Analysis position: 3760 -> 3765
use_tmi_words_value: [3 7 3 2 4]
similar_sentence_value [1862  234   90  344 1343]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 3765 -> 3770
use_tmi_words_value: [ 0  1 12  5  5]
similar_sentence_value [   0  121  930  838 1511]
commissional_words_value: [0 0 0 1 0]

Data cast
Analysis position: 3770 -> 3775
use_tmi_words_value: [6 5 3 3 7]
similar_sentence_value [187 427 445 403 452]
commissional_words_value: [0 1 0 1 0]

Data cast
Analysis position: 3775 -> 3780
use_tmi_words_value: [ 1 14  2  4  3]
s

similar_sentence_value [ 417 1436  177  966 5272]
commissional_words_value: [0 0 0 1 0]

Data cast
Analysis position: 4000 -> 4005
use_tmi_words_value: [13 12  6 34 25]
similar_sentence_value [2485 1968  106 2764 3018]
commissional_words_value: [1 0 1 1 0]

Data cast
Analysis position: 4005 -> 4010
use_tmi_words_value: [26 28 27  6  3]
similar_sentence_value [4121 2209 4059  467  158]
commissional_words_value: [1 1 0 1 0]

Data cast
Analysis position: 4010 -> 4015
use_tmi_words_value: [ 7  1  1 28 20]
similar_sentence_value [2704  722 1508 1428 6325]
commissional_words_value: [1 0 0 1 0]

Data cast
Analysis position: 4015 -> 4020
use_tmi_words_value: [23 25 35  4 20]
similar_sentence_value [4363 3849 4120  464 3897]
commissional_words_value: [1 1 1 1 1]

Data cast
Analysis position: 4020 -> 4025
use_tmi_words_value: [35  1  1 10  1]
similar_sentence_value [2417 4553  861 3898  473]
commissional_words_value: [1 1 0 0 0]

Data cast
Analysis position: 4025 -> 4030
use_tmi_words_value: [25

similar_sentence_value [3128 3537 2983 1450  161]
commissional_words_value: [1 1 0 0 1]

Data cast
Analysis position: 4250 -> 4255
use_tmi_words_value: [21 35 20 37 15]
similar_sentence_value [5338 3650 2474 2670 1962]
commissional_words_value: [1 1 1 1 1]

Data cast
Analysis position: 4255 -> 4260
use_tmi_words_value: [36 25  8  6 24]
similar_sentence_value [5974 3132  255   60 2852]
commissional_words_value: [1 0 0 0 1]

Data cast
Analysis position: 4260 -> 4265
use_tmi_words_value: [27 37 35 34 24]
similar_sentence_value [1436 2561 3064  934 3154]
commissional_words_value: [0 1 1 1 1]

Data cast
Analysis position: 4265 -> 4270
use_tmi_words_value: [1 1 0 1 2]
similar_sentence_value [ 491   66    0 1177  338]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 4270 -> 4275
use_tmi_words_value: [2 0 2 2 1]
similar_sentence_value [803   0 311 145 293]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 4275 -> 4280
use_tmi_words_value: [ 1 25  1  3  8]
s

similar_sentence_value [ 569  197  327 2327  270]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 4500 -> 4505
use_tmi_words_value: [0 2 1 6 5]
similar_sentence_value [  0 320  72 647  87]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 4505 -> 4510
use_tmi_words_value: [ 6 12  0  0 11]
similar_sentence_value [ 211   54    0    0 1651]
commissional_words_value: [0 1 0 0 1]

Data cast
Analysis position: 4510 -> 4515
use_tmi_words_value: [0 0 0 0 5]
similar_sentence_value [  0   0 332   0 152]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 4515 -> 4520
use_tmi_words_value: [ 0 10  7  0  0]
similar_sentence_value [  0 453 227   0   0]
commissional_words_value: [0 1 1 0 0]

Data cast
Analysis position: 4520 -> 4525
use_tmi_words_value: [ 0  0  0 27 31]
similar_sentence_value [   0    0    0 2187 2151]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 4525 -> 4530
use_tmi_words_value: [ 9 10 10 10  9]
similar_sen

similar_sentence_value [ 429 2851  191 2887 1457]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 4750 -> 4755
use_tmi_words_value: [4 2 4 0 1]
similar_sentence_value [ 370  755 1208    0 3560]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 4755 -> 4760
use_tmi_words_value: [12 10  4  4  7]
similar_sentence_value [1041  952  137   95  171]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 4760 -> 4765
use_tmi_words_value: [3 2 1 5 7]
similar_sentence_value [598 101 896 942   8]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 4765 -> 4770
use_tmi_words_value: [ 2  3 10  4  4]
similar_sentence_value [  88  362 2215  270  456]
commissional_words_value: [0 1 1 0 0]

Data cast
Analysis position: 4770 -> 4775
use_tmi_words_value: [4 6 1]
similar_sentence_value [3387  608  140]
commissional_words_value: [0 1 0]

Data cast
Analysis finish


# 분석 결과 확인

In [11]:
analysis_result = pd.read_csv("submission.csv")
analysis_result = analysis_result.fillna(0)

In [12]:
analysis_result

,url,use_tmi_words_value,similar_sentence_value,commissional_words_value
0,https://blog.naver.com/apfjtzzz/222354696565,12,973,0
1,https://blog.naver.com/fa1772/222377394130,36,5517,1
2,https://blog.naver.com/1457dudgus/222565835669,8,816,1
3,https://blog.naver.com/help1207/222548639407,15,2263,1
4,https://blog.naver.com/sweets9/222548166927,9,719,1
...,...,...,...,...
4768,https://blog.naver.com/ejkim0422/221992057540,4,270,0
4769,https://blog.naver.com/kfs97/221734966254,4,456,0
4770,https://blog.naver.com/free6568/221779346538,4,3387,0
4771,https://blog.naver.com/lsk929292/222520018600,6,608,1


In [13]:
analysis_result.shape

(4773, 4)

# 분석 결과 정규화

In [14]:
def normalization(data_set):
    x_max = max(data_set)
    x_min = min(data_set)
    
    # 최댓값 최솟값이 둘다 0이라면 어차피, 0으로 이루어진 리스트이므로 그대로 반환
    if(x_max - x_min) == 0:
        return data_set
    
    result = np.array((data_set - x_min) / (x_max - x_min))
    
    return result.tolist()

In [15]:
analysis_result["use_tmi_words_value"] = normalization(analysis_result["use_tmi_words_value"])
analysis_result["similar_sentence_value"] = normalization(analysis_result["similar_sentence_value"])
analysis_result["commissional_words_value"] = normalization(analysis_result["commissional_words_value"])

analysis_result

,url,use_tmi_words_value,similar_sentence_value,commissional_words_value
0,https://blog.naver.com/apfjtzzz/222354696565,0.011070,0.058142,0.0
1,https://blog.naver.com/fa1772/222377394130,0.033210,0.329668,1.0
2,https://blog.naver.com/1457dudgus/222565835669,0.007380,0.048760,1.0
3,https://blog.naver.com/help1207/222548639407,0.013838,0.135226,1.0
4,https://blog.naver.com/sweets9/222548166927,0.008303,0.042964,1.0
...,...,...,...,...
4768,https://blog.naver.com/ejkim0422/221992057540,0.003690,0.016134,0.0
4769,https://blog.naver.com/kfs97/221734966254,0.003690,0.027248,0.0
4770,https://blog.naver.com/free6568/221779346538,0.003690,0.202390,0.0
4771,https://blog.naver.com/lsk929292/222520018600,0.005535,0.036331,1.0


# k-means 알고리즘을 통한 classification

In [16]:
from sklearn.cluster import KMeans
import matplotlib.pyplot  as plt

# 특징 요소 분리
feature = analysis_result[["use_tmi_words_value", "similar_sentence_value", "commissional_words_value"]]

# 세개의 군집으로 분리
# 각 군집은, 바이럴 데이터 정도 낮음, 중간, 높음을 의미함
model = KMeans(n_clusters=3,algorithm='auto')
model.fit(feature)
predict = pd.DataFrame(model.predict(feature))
predict.columns=['predict']
print(predict['predict'])

0       0
1       1
2       1
3       1
4       1
       ..
4768    0
4769    0
4770    2
4771    1
4772    0
Name: predict, Length: 4773, dtype: int32


In [17]:
# submission.csv에 기록
f = pd.read_csv("submission.csv")
f = f.fillna(0)
f['class'] = predict['predict']

f.to_csv('submission.csv', mode='w')

# 모듈 종료

In [18]:
print("text_module_finish")

text_module_finish
